In [1]:
# PACKAGES
import numpy as np  # For numerical operations
import pandas as pd  # For data manipulation and analysis
import matplotlib.pyplot as plt  # For plotting graphs
from sklearn.metrics import mean_squared_error, r2_score  # For model evaluation metrics
from sklearn.linear_model import LinearRegression  # For linear regression modeling
from sklearn.preprocessing import PolynomialFeatures  # For polynomial regression modeling
from statsmodels.tsa.api import SimpleExpSmoothing, ExponentialSmoothing  # For time series forecasting
from tabulate import tabulate  # For tabular display of data
import os  # For interacting with the operating system
import json  # For handling JSON data

# DATA
file_path = 'data2.json'  # Path to the JSON data file
with open(file_path, 'r') as f:  # Open and read the JSON file
    json_data = json.load(f)  # Load data into a Python dictionary
data = pd.DataFrame(json_data)  # Convert dictionary to a pandas DataFrame
display(data)

,P/N,D-1,D-2,D-3,D-4,D-5,D-6,D-7,D-8,D-9,D-10,D-11,D-12,D-13
0,ZZ01,23,22,21,20,19,18,17,16,15,14,13,12,11
1,ZZ02,23,33,22,11,90,89,78,67,56,45,34,23,22
2,129888,10,6,2,2,12,0,8,4,14,8,2,8,10
3,130083,4,18,6,6,10,24,6,16,12,18,0,6,6
4,131584,2,7,6,0,2,2,3,7,8,8,0,1,1
5,132770,2,0,0,0,2,0,0,2,0,0,0,0,0
6,205230,60,60,60,60,60,102,0,94,26,26,94,34,34
7,205401,0,120,96,1,98,2,0,66,1,49,144,0,0
8,3631163,0,128,64,64,64,128,64,128,64,128,0,64,64
9,4024792,0,0,2,1,0,0,0,2,0,0,0,1,0


In [2]:

# FUNCTION TO PROCESS DATA FOR A SPECIFIC P/N
def process_pn(data, pn_to_filter):
    # Filter data for the specified part number (P/N)
    filtered_data = data[data['P/N'] == pn_to_filter]
    formatted_data = []  # Initialize an empty list for formatted data
    # print(filtered_data)

    # Loop through filtered rows to reformat data
    for index, row in filtered_data.iterrows():
        # Extract quantities from D-2 to D-13 and reverse the order
        quantities = [row[f'D-{i}'] for i in range(2, 14)][::-1]  # Reverse here ## Kenapa 1 nya nggak di ambil?
        # print(quantities)

        # Create formatted data with period and quantity
        for period, quantity in enumerate(quantities, start=1):
            formatted_data.append({
                'P/N': str(row['P/N']),  # Ensure P/N is stored as a string
                'Period': period,  # Add period index
                'Quantity': quantity if not pd.isna(quantity) else None  # Handle missing data
            })

    # Convert the list of dictionaries to a DataFrame
    formatted_df = pd.DataFrame(formatted_data)
    # display(formatted_df)

    # Calculate mean and standard deviation, and set upper bound
    mean_quantity = formatted_df['Quantity'].mean()
    std_quantity = formatted_df['Quantity'].std()


    LB = 0  # Lower bound is set to 0
    UB = mean_quantity + 1.5 * std_quantity  # Calculate upper bound as mean + 1.5 * std deviation

    # round all data
    mean_quantity = round(mean_quantity, 4)  # Round mean value
    std_quantity = round(std_quantity, 4)  # Round standard deviation
    UB = round(UB, 4)  # Round upper bound

    # Print metrics in a table format
    table = [
        ["Mean Quantity", mean_quantity],
        ["Standard Deviation", std_quantity],
        ["Upper Bound (UB)", UB]
    ]
    print(tabulate(table, headers=["Metric", "Value"], tablefmt="pretty"))

    # Clip quantities to upper bound (replace values above UB with UB)
    formatted_df['Quantity'] = np.clip(formatted_df['Quantity'], LB, UB)
    return formatted_df  # Return the processed DataFrame

# SELECT P/N TO FILTER
pn_to_filter = "ZZ01"  # Change this to filter data for a different part number
formatted_df = process_pn(data, pn_to_filter)  # Process data for the selected P/N
display(formatted_df)

+--------------------+---------+
|       Metric       |  Value  |
+--------------------+---------+
|   Mean Quantity    |  16.5   |
| Standard Deviation | 3.6056  |
|  Upper Bound (UB)  | 21.9083 |
+--------------------+---------+


,P/N,Period,Quantity
0,ZZ01,1,11.0000
1,ZZ01,2,12.0000
2,ZZ01,3,13.0000
3,ZZ01,4,14.0000
4,ZZ01,5,15.0000
5,ZZ01,6,16.0000
6,ZZ01,7,17.0000
7,ZZ01,8,18.0000
8,ZZ01,9,19.0000
9,ZZ01,10,20.0000


In [3]:

# MOVING AVERAGE
# Initialize Period column and moving average prediction column
formatted_df['Period'] = np.arange(1, len(formatted_df) + 1)
formatted_df['MA_Prediction'] = np.nan

# Calculate moving average for each period
for i in range(1, len(formatted_df) + 1):
    formatted_df.loc[formatted_df['Period'] == i, 'MA_Prediction'] = formatted_df['Quantity'].iloc[:i].mean()

display(formatted_df)

# If there are 12 periods, predict the 13th period as the mean of the last 12 quantities
if len(formatted_df) == 12:
    last_12_mean = formatted_df['Quantity'].iloc[-12:].mean()
    formatted_df = pd.concat([
        formatted_df,
        pd.DataFrame({'Period': [13], 'MA_Prediction': [last_12_mean], 'Quantity': [np.nan]})
    ], ignore_index=True)
    
display(formatted_df)
    

,P/N,Period,Quantity,MA_Prediction
0,ZZ01,1,11.0000,11.000000
1,ZZ01,2,12.0000,11.500000
2,ZZ01,3,13.0000,12.000000
3,ZZ01,4,14.0000,12.500000
4,ZZ01,5,15.0000,13.000000
5,ZZ01,6,16.0000,13.500000
6,ZZ01,7,17.0000,14.000000
7,ZZ01,8,18.0000,14.500000
8,ZZ01,9,19.0000,15.000000
9,ZZ01,10,20.0000,15.500000


,P/N,Period,Quantity,MA_Prediction
0,ZZ01,1,11.0000,11.000000
1,ZZ01,2,12.0000,11.500000
2,ZZ01,3,13.0000,12.000000
3,ZZ01,4,14.0000,12.500000
4,ZZ01,5,15.0000,13.000000
5,ZZ01,6,16.0000,13.500000
6,ZZ01,7,17.0000,14.000000
7,ZZ01,8,18.0000,14.500000
8,ZZ01,9,19.0000,15.000000
9,ZZ01,10,20.0000,15.500000


In [4]:

# WEIGHTED MOVING AVERAGE (WMA)
def calculate_exponential_weighted_moving_average(values, alpha_wma=0.3):
    # Calculate exponential weighted moving average with specified alpha
    return values.ewm(alpha=alpha_wma, adjust=False).mean()

alpha_wma = 0.3  # Smoothing parameter for WMA
formatted_df['Smoothed_Weighted_Prediction'] = calculate_exponential_weighted_moving_average(
    formatted_df['Quantity'], alpha_wma
)
display(formatted_df)

,P/N,Period,Quantity,MA_Prediction,Smoothed_Weighted_Prediction
0,ZZ01,1,11.0000,11.000000,11.000000
1,ZZ01,2,12.0000,11.500000,11.300000
2,ZZ01,3,13.0000,12.000000,11.810000
3,ZZ01,4,14.0000,12.500000,12.467000
4,ZZ01,5,15.0000,13.000000,13.226900
5,ZZ01,6,16.0000,13.500000,14.058830
6,ZZ01,7,17.0000,14.000000,14.941181
7,ZZ01,8,18.0000,14.500000,15.858827
8,ZZ01,9,19.0000,15.000000,16.801179
9,ZZ01,10,20.0000,15.500000,17.760825


In [8]:

# LINEAR REGRESSION
# Prepare data for linear regression
valid_data = formatted_df.dropna(subset=['Quantity'])
X = valid_data[['Period']]  # Independent variable
y = valid_data['Quantity']  # Dependent variable
print(X)
print(y)
model = LinearRegression()  # Initialize linear regression model
model.fit(X, y)  # Fit model to the data
formatted_df['Linear_Regression_Prediction'] = model.predict(formatted_df[['Period']])  # Predict for all periods
display(formatted_df)

    Period
0        1
1        2
2        3
3        4
4        5
5        6
6        7
7        8
8        9
9       10
10      11
11      12
0     11.0000
1     12.0000
2     13.0000
3     14.0000
4     15.0000
5     16.0000
6     17.0000
7     18.0000
8     19.0000
9     20.0000
10    21.0000
11    21.9083
Name: Quantity, dtype: float64


,P/N,Period,Quantity,MA_Prediction,Smoothed_Weighted_Prediction,Linear_Regression_Prediction
0,ZZ01,1,11.0000,11.000000,11.000000,11.011756
1,ZZ01,2,12.0000,11.500000,11.300000,12.008229
2,ZZ01,3,13.0000,12.000000,11.810000,13.004703
3,ZZ01,4,14.0000,12.500000,12.467000,14.001176
4,ZZ01,5,15.0000,13.000000,13.226900,14.997649
5,ZZ01,6,16.0000,13.500000,14.058830,15.994122
6,ZZ01,7,17.0000,14.000000,14.941181,16.990595
7,ZZ01,8,18.0000,14.500000,15.858827,17.987068
8,ZZ01,9,19.0000,15.000000,16.801179,18.983541
9,ZZ01,10,20.0000,15.500000,17.760825,19.980014


In [7]:

# POLYNOMIAL REGRESSION (DEGREE 2)
degree_2 = 2  # Degree of the polynomial
poly = PolynomialFeatures(degree=degree_2)  # Create polynomial features
X_poly = poly.fit_transform(X)  # Transform input features
poly_model = LinearRegression()  # Initialize linear regression model
poly_model.fit(X_poly, y)  # Fit polynomial model
X_all_poly = poly.transform(formatted_df[['Period']])  # Transform all periods
formatted_df['Polynomial_Regression_Prediction_Degree_2'] = poly_model.predict(X_all_poly)  # Predict values

# POLYNOMIAL REGRESSION (DEGREE 3)
degree_3 = 3  # Degree of the polynomial
poly = PolynomialFeatures(degree=degree_3)  # Create polynomial features
X_poly = poly.fit_transform(X)  # Transform input features
poly_model = LinearRegression()  # Initialize linear regression model
poly_model.fit(X_poly, y)  # Fit polynomial model
X_all_poly = poly.transform(formatted_df[['Period']])  # Transform all periods
formatted_df['Polynomial_Regression_Prediction_Degree_3'] = poly_model.predict(X_all_poly)  # Predict values

# SINGLE EXPONENTIAL SMOOTHING (SES)
alpha_ses = 0.8  # Smoothing parameter for SES
def apply_ses(values, alpha=alpha_ses):
    # Apply single exponential smoothing to the data
    ses_model = SimpleExpSmoothing(values).fit(smoothing_level=alpha, optimized=False)
    return ses_model.fittedvalues

# DOUBLE EXPONENTIAL SMOOTHING (DES)
beta_des = 0.3  # Trend smoothing parameter for DES
def apply_des(values, alpha=alpha_ses, beta=beta_des):
    # Apply double exponential smoothing to the data
    des_model = ExponentialSmoothing(values, trend='add', seasonal=None).fit(smoothing_level=alpha, smoothing_trend=beta, optimized=False)
    return des_model.fittedvalues

formatted_df['SES_Prediction'] = apply_ses(formatted_df['Quantity'], alpha=alpha_ses)  # Apply SES
formatted_df['DES_Prediction'] = apply_des(formatted_df['Quantity'], alpha=alpha_ses, beta=beta_des)  # Apply DES

display(formatted_df)

,P/N,Period,Quantity,MA_Prediction,Smoothed_Weighted_Prediction,Linear_Regression_Prediction,Polynomial_Regression_Prediction_Degree_2,Polynomial_Regression_Prediction_Degree_3,SES_Prediction,DES_Prediction
0,ZZ01,1,11.0000,11.000000,11.000000,11.011756,10.988663,11.008062,11.000000,11.000000
1,ZZ01,2,12.0000,11.500000,11.300000,12.008229,11.997733,11.995969,11.000000,12.000000
2,ZZ01,3,13.0000,12.000000,11.810000,13.004703,13.004283,12.991938,11.800000,13.000000
3,ZZ01,4,14.0000,12.500000,12.467000,14.001176,14.008313,13.993618,12.760000,14.000000
4,ZZ01,5,15.0000,13.000000,13.226900,14.997649,15.009825,14.998656,13.752000,15.000000
5,ZZ01,6,16.0000,13.500000,14.058830,15.994122,16.008817,16.004703,14.750400,16.000000
6,ZZ01,7,17.0000,14.000000,14.941181,16.990595,17.005290,17.009405,15.750080,17.000000
7,ZZ01,8,18.0000,14.500000,15.858827,17.987068,17.999244,18.010413,16.750016,18.000000
8,ZZ01,9,19.0000,15.000000,16.801179,18.983541,18.990679,19.005374,17.750003,19.000000
9,ZZ01,10,20.0000,15.500000,17.760825,19.980014,19.979594,19.991938,18.750001,20.000000


In [9]:

# RMSE & R2 METRICS
results = []  # Initialize list to store results
# Evaluate models for RMSE and R2
for column in ['MA_Prediction', 'Smoothed_Weighted_Prediction', 'Linear_Regression_Prediction',
               'Polynomial_Regression_Prediction_Degree_2', 'Polynomial_Regression_Prediction_Degree_3',
               'SES_Prediction', 'DES_Prediction']:
    valid_data = formatted_df.dropna(subset=['Quantity', column])  # Filter valid data
    if valid_data.empty:  # Skip if no valid data
        print(f"Skipping {column} due to lack of valid data.")
        continue
    try:
        rmse = np.sqrt(mean_squared_error(valid_data['Quantity'], valid_data[column]))  # Calculate RMSE
        r2 = r2_score(valid_data['Quantity'], valid_data[column])  # Calculate R2
        results.append({'Model': column, 'RMSE': rmse, 'R2': r2})  # Append results
    except Exception as e:
        print(f"Error evaluating {column}: {e}")  # Handle errors

results_df = pd.DataFrame(results)  # Convert results to a DataFrame
display(valid_data)
display(results_df)

,P/N,Period,Quantity,MA_Prediction,Smoothed_Weighted_Prediction,Linear_Regression_Prediction,Polynomial_Regression_Prediction_Degree_2,Polynomial_Regression_Prediction_Degree_3,SES_Prediction,DES_Prediction
0,ZZ01,1,11.0000,11.000000,11.000000,11.011756,10.988663,11.008062,11.000000,11.0
1,ZZ01,2,12.0000,11.500000,11.300000,12.008229,11.997733,11.995969,11.000000,12.0
2,ZZ01,3,13.0000,12.000000,11.810000,13.004703,13.004283,12.991938,11.800000,13.0
3,ZZ01,4,14.0000,12.500000,12.467000,14.001176,14.008313,13.993618,12.760000,14.0
4,ZZ01,5,15.0000,13.000000,13.226900,14.997649,15.009825,14.998656,13.752000,15.0
5,ZZ01,6,16.0000,13.500000,14.058830,15.994122,16.008817,16.004703,14.750400,16.0
6,ZZ01,7,17.0000,14.000000,14.941181,16.990595,17.005290,17.009405,15.750080,17.0
7,ZZ01,8,18.0000,14.500000,15.858827,17.987068,17.999244,18.010413,16.750016,18.0
8,ZZ01,9,19.0000,15.000000,16.801179,18.983541,18.990679,19.005374,17.750003,19.0
9,ZZ01,10,20.0000,15.500000,17.760825,19.980014,19.979594,19.991938,18.750001,20.0


,Model,RMSE,R2
0,MA_Prediction,3.234996,0.115611
1,Smoothed_Weighted_Prediction,1.823989,0.718848
2,Linear_Regression_Prediction,0.022229,0.999958
3,Polynomial_Regression_Prediction_Degree_2,0.017823,0.999973
4,Polynomial_Regression_Prediction_Degree_3,0.013016,0.999986
5,SES_Prediction,1.163705,0.885559
6,DES_Prediction,0.026472,0.999941


In [ ]:

# BEST MODEL SELECTION
best_model_row = results_df.loc[results_df['R2'].idxmax()]  # Select best model by highest R2
best_model_name = best_model_row['Model']  # Best model name
best_model_r2 = best_model_row['R2']  # Best model R2 value
next_period_prediction = formatted_df.loc[formatted_df['Period'] == 13, best_model_name].values[0]  # Predict next period

# PLOT ALL GRAPHS
plt.figure(figsize=(12, 8))  # Set figure size
plt.plot(formatted_df['Period'], formatted_df['Quantity'], label='Data Historis', marker='o', color='black')  # Plot historical data
plt.plot(formatted_df['Period'], formatted_df['MA_Prediction'], label='Prediksi MA', linestyle='--', color='orange')  # Plot MA predictions
plt.plot(formatted_df['Period'], formatted_df['Smoothed_Weighted_Prediction'], label='Prediksi WMA', linestyle='--', color='blue')  # Plot WMA predictions
plt.plot(formatted_df['Period'], formatted_df['Linear_Regression_Prediction'], label='Prediksi Regresi Linear', linestyle='--', color='green')  # Plot linear regression predictions
plt.plot(formatted_df['Period'], formatted_df['Polynomial_Regression_Prediction_Degree_2'], label='Prediksi Regresi Polinomial Derajat 2', linestyle='--', color='red')  # Plot polynomial regression (degree 2) predictions
plt.plot(formatted_df['Period'], formatted_df['Polynomial_Regression_Prediction_Degree_3'], label='Prediksi Regresi Polinomial Derajat 3', linestyle='--', color='purple')  # Plot polynomial regression (degree 3) predictions
plt.plot(formatted_df['Period'], formatted_df['SES_Prediction'], label='Prediksi SES', linestyle='--', color='magenta')  # Plot SES predictions
plt.plot(formatted_df['Period'], formatted_df['DES_Prediction'], label='Prediksi DES', linestyle='--', color='yellow')  # Plot DES predictions
plt.xlabel('Period')  # Set x-axis label
plt.ylabel('Quantity')  # Set y-axis label
plt.title(f'Prediksi Quantity untuk P/N {pn_to_filter}')  # Set plot title
plt.legend()  # Add legend
plt.grid(False)  # Disable grid
plt.show()  # Display the plot

# PLOT BEST MODEL
plt.figure(figsize=(12, 8))  # Set figure size
plt.plot(formatted_df['Period'], formatted_df['Quantity'], label='Data Historis', marker='o', color='black')  # Plot historical data
plt.plot(formatted_df['Period'], formatted_df[best_model_name], label=f'Best Model: {best_model_name}', linestyle='-', color='blue')  # Plot best model predictions
plt.xlabel('Period')  # Set x-axis label
plt.ylabel('Quantity')  # Set y-axis label
plt.title(f'Prediksi menggunakan {best_model_name} untuk 1 bulan kedepan')  # Set plot title
plt.legend()  # Add legend
plt.grid(False)  # Disable grid
plt.show()  # Display the plot

# DISPLAY DATAFRAMES
print(tabulate(formatted_df, headers='keys', tablefmt='fancy_grid'))  # Print formatted data
print(tabulate(results_df.sort_values(by='R2', ascending=False), headers='keys', tablefmt='fancy_grid'))  # Print sorted results

# DISPLAY BEST MODEL SUMMARY
table_data = [
    ["P/N", pn_to_filter],
    ["Best Model", best_model_name],
    ["Best R-squared", f"{best_model_r2:.4f}"],
    ["Prediction for 13th Period", f"{next_period_prediction:.0f}"]
]
print(tabulate(table_data, headers=["Metric", "Value"], tablefmt="fancy_grid"))  # Print summary
display(formatted_df)
# EXPORT RESULTS TO JSON
output_file_path = 'results.json'  # Path to output file
formatted_df.to_json(output_file_path, orient='records', indent=4)  # Save DataFrame to JSON
